In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools 

In [ ]:
def genState(vars,s,i,n):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i), BVType(n))
    state['pc'] = Symbol('pc'+'!'+s+str(i), INT)
    state['q'] = Symbol('q'+'!'+s+str(i), BVType(n))
    return state

def init(state, a, b, n):
    return And(
        Equals(state['r'], a),
        Equals(state['r2'], b),
        Equals(state['s'], BV(1, n)),
        Equals(state['s2'], BV(0, n)),
        Equals(state['t'], BV(0, n)),
        Equals(state['t2'], BV(1, n)),
        BVUGT(a, BV(0, n)),
        BVUGT(b, BV(0, n)),
        Equals(state['pc'], Int(0))
    )

def error1(state):
    return Equals(state['pc'], Int(3))

def trans1(curr, prox):
    n = curr['r'].symbol_type().width

    # Cálculo de q (divisão inteira entre r e r2)
    #q = BVUDiv(curr['r'], curr['r2'])
    
    
    t_main = And(
        Or(
            Equals(curr['pc'], Int(0)),
            Equals(curr['pc'], Int(1))
        ),
        Not(Equals(curr['r2'], BV(0, n))),
        Equals(prox['pc'], Int(1) - curr['pc']),  
        Equals(prox['r'], curr['r2']),
        Equals(prox['r2'], BVSub(curr['r'], BVMul(curr['q'], curr['r2']))),
        Equals(prox['s'], curr['s2']),
        Equals(prox['s2'], BVSub(curr['s'], BVMul(curr['q'], curr['s2']))),
        Equals(prox['t'], curr['t2']),
        Equals(prox['t2'], BVSub(curr['t'], BVMul(curr['q'], curr['t2']))),
        Equals(curr['q'], BVUDiv(curr['r'], curr['r2']))
    )


    erro = And(
        Equals(curr['pc'], Int(1)),
        Equals(curr['r'], BV(0, n)),
        Equals(prox['pc'], Int(3))  
    )
    
    erro_overflow = And(
        Equals(curr['pc'], Int(1)),
        Or(
            BVUGT(curr['r'], BV(2**n - 1, n)),
            BVUGT(curr['r2'], BV(2**n - 1, n)),
            BVUGT(curr['s'], BV(2**n - 1, n)),
            BVUGT(curr['s2'], BV(2**n - 1, n)),
            BVUGT(curr['t'], BV(2**n - 1, n)),
            BVUGT(curr['t2'], BV(2**n - 1, n)),
            BVUGT(curr['q'], BV(2**n - 1, n)),
        ),
        Equals(prox['pc'], Int(3))  
    )

    final = And(
        Equals(curr['pc'], Int(1)),
        Equals(curr['r2'], BV(0, n)),
        Equals(prox['pc'], Int(2)),
        Equals(prox['r'], curr['r']),  
        Equals(prox['s'], curr['s']),  
        Equals(prox['t'], curr['t']),
        Equals(prox['r2'], curr['r2']), 
        Equals(prox['s2'], curr['s2']), 
        Equals(prox['t2'], curr['t2']),
        Equals(prox['q'], curr['q']),
        
    )
    return Or(t_main, erro, erro_overflow, final)
    
def genTrace(vars, init, trans, error, n, N):
    with Solver(name="z3") as s:
        X = [genState(vars, 'X', i, n) for i in range(N+1)]
        a = Symbol('a',BVType(n))
        b = Symbol('b',BVType(n))
        I = init(X[0], a, b, n)
        traces = [trans(X[i], X[i+1]) for i in range(N)]
        
        final_state = Equals(X[N]['pc'], Int(2))
        
        if s.solve([I, And(traces), final_state]):
            model = s.get_model()
            for i in range(N+1):
                print(f"Estado X{i}:")
                for v in X[i]:
                    value = model[X[i][v]].constant_value()
                    print(f"        {v} = {value}")
        else:
            print("No model found")

vars = ['r', 'r2', 's', 's2', 't', 't2']
genTrace(vars, init, trans1, error1, 8, 8)

Estado X0:
        r = 73
        r2 = 89
        s = 1
        s2 = 0
        t = 0
        t2 = 1
        pc = 0
        q = 0
Estado X1:
        r = 89
        r2 = 73
        s = 0
        s2 = 1
        t = 1
        t2 = 0
        pc = 1
        q = 1
Estado X2:
        r = 73
        r2 = 16
        s = 1
        s2 = 255
        t = 0
        t2 = 1
        pc = 0
        q = 4
Estado X3:
        r = 16
        r2 = 9
        s = 255
        s2 = 5
        t = 1
        t2 = 252
        pc = 1
        q = 1
Estado X4:
        r = 9
        r2 = 7
        s = 5
        s2 = 250
        t = 252
        t2 = 5
        pc = 0
        q = 1
Estado X5:
        r = 7
        r2 = 2
        s = 250
        s2 = 11
        t = 5
        t2 = 247
        pc = 1
        q = 3
Estado X6:
        r = 2
        r2 = 1
        s = 11
        s2 = 217
        t = 247
        t2 = 32
        pc = 0
        q = 2
Estado X7:
        r = 1
        r2 = 0
        s = 217
        s2 = 89
        t =

In [3]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def invert(trans):
    return lambda curr, prox: trans(prox, curr)

def genState2(vars, s, i, n):
    state = {}
    for v in vars:
        state[v] = Symbol(v + '!' + s + str(i), BVType(n))
    #state['pc'] = Symbol('pc' + '!' + s + str(i), BVType(n))  # Alterado para BVType
    state['q'] = Symbol('q' + '!' + s + str(i), BVType(n))
    return state

def init2(state, a, b, n):
    return And(
        Equals(state['r'], a),
        Equals(state['r2'], b),
        Equals(state['s'], BV(1, n)),
        Equals(state['s2'], BV(0, n)),
        Equals(state['t'], BV(0, n)),
        Equals(state['t2'], BV(1, n)),
        BVUGT(a, BV(0, n)),
        BVUGT(b, BV(0, n)),
        Equals(state['pc2'], BV(0, n))  # Alterado para BV
    )

def error2(state, n):
    return Equals(state['pc2'], BV(3, n))  # Alterado para BV

def trans2(curr, prox):
    n = curr['r'].symbol_type().width

    # Transições principais
    t_main = And(
        Or(
            Equals(curr['pc2'], BV(0, n)),
            Equals(curr['pc2'], BV(1, n))
        ),
        Not(Equals(curr['r2'], BV(0, n))),
        Equals(prox['pc2'], BV(1, n) - curr['pc2']),  
        Equals(prox['r'], curr['r2']),
        Equals(prox['r2'], BVSub(curr['r'], BVMul(curr['q'], curr['r2']))),
        Equals(prox['s'], curr['s2']),
        Equals(prox['s2'], BVSub(curr['s'], BVMul(curr['q'], curr['s2']))),
        Equals(prox['t'], curr['t2']),
        Equals(prox['t2'], BVSub(curr['t'], BVMul(curr['q'], curr['t2']))),
        Equals(curr['q'], BVUDiv(curr['r'], curr['r2']))
    )

    # Erro
    erro = And(
        Equals(curr['pc2'], BV(1, n)),
        Equals(curr['r'], BV(0, n)),
        Equals(prox['pc2'], BV(3, n))  
    )

    # Overflow
    erro_overflow = And(
        Equals(curr['pc2'], BV(1, n)),
        Or(
            BVUGT(curr['r'], BV(2**n - 1, n)),
            BVUGT(curr['r2'], BV(2**n - 1, n)),
            BVUGT(curr['s'], BV(2**n - 1, n)),
            BVUGT(curr['s2'], BV(2**n - 1, n)),
            BVUGT(curr['t'], BV(2**n - 1, n)),
            BVUGT(curr['t2'], BV(2**n - 1, n)),
            BVUGT(curr['q'], BV(2**n - 1, n)),
        ),
        Equals(prox['pc2'], BV(3, n))  
    )

    # Estado final
    final = And(
        Equals(curr['pc2'], BV(1, n)),
        Equals(curr['r2'], BV(0, n)),
        Equals(prox['pc2'], BV(2, n)),
        Equals(prox['r'], curr['r']),  
        Equals(prox['s'], curr['s']),  
        Equals(prox['t'], curr['t']),
        Equals(prox['r2'], curr['r2']), 
        Equals(prox['s2'], curr['s2']), 
        Equals(prox['t2'], curr['t2']),
        Equals(prox['q'], curr['q']),
    )
    return Or(t_main, erro, erro_overflow, final)


def model_checking(vars, init, trans, error, bits, N, M):
    with Solver(name="z3") as solver:
        a = Symbol('a', BVType(bits))
        b = Symbol('b', BVType(bits))
        X = [genState2(vars, "X", i, bits) for i in range(N+1)]
        Y = [genState2(vars, "Y", i, bits) for i in range(M+1)]
        transt = invert(trans)

        order = sorted([(x, y) for x in range(1, N+1) for y in range(1, M+1)], key=lambda tup: tup[0] + tup[1])

        for (n, m) in order:
            I = init(X[0], a, b, bits)  # init2
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])  # trans2
            Rn = And(I, Tn)

            E = error(Y[0], bits)  # error2
            Bm = And([transt(Y[i], Y[i+1]) for i in range(m)])
            Um = And(E, Bm)

            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro")
                return
            else:
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B)
                if C is None:
                    print("> O interpolante é None")
                    break
                
                C0 = rename(C, X[0])
                T = trans(X[0], X[1])  # trans2
                C1 = rename(C, X[1])

                if not solver.solve([C0, T, Not(C1)]):
                    print("> O sistema é seguro")
                    return
                else:
                    S = rename(C, X[n])
                    while True:
                        T = trans(X[n], Y[m])  # trans2
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante")
                            break
                        else:
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                                print("> O sistema é seguro")
                                return
                            else:
                                S = Or(S, Cn)

    print("> Não foi provada a segurança ou insegurança do sistema")


vars = ['r', 'r2', 's', 's2', 't', 't2', 'pc2']
model_checking(vars, init2, trans2, error2, 8, 8, 8)


> O sistema é seguro


In [4]:
def genState(vars, s, i, n):
    state = {}
    for v in vars:
        state[v] = Symbol(v + '!' + s + str(i), BVType(n))
    state['pc'] = Symbol('pc' + '!' + s + str(i), INT)
    state['q'] = Symbol('q' + '!' + s + str(i), BVType(n))
    return state

def init(state, a, b, n):
    return And(
        Equals(state['r'], a),
        Equals(state['r2'], b),
        Equals(state['s'], BV(1, n)),
        Equals(state['s2'], BV(0, n)),
        Equals(state['t'], BV(0, n)),
        Equals(state['t2'], BV(1, n)),
        BVUGT(a, BV(0, n)),  # Unsigned comparison
        BVUGT(b, BV(0, n)),  # Unsigned comparison
        Equals(state['pc'], Int(0))
    )

def error1(state):
    return Equals(state['pc'], Int(3))

def trans1(curr, prox):
    n = curr['r'].symbol_type().width

    t_main = And(
        Or(
            Equals(curr['pc'], Int(0)),
            Equals(curr['pc'], Int(1))
        ),
        Not(Equals(curr['r2'], BV(0, n))),
        Equals(prox['pc'], Int(1) - curr['pc']),
        Equals(prox['r'], curr['r2']),
        Equals(prox['r2'], BVSub(curr['r'], BVUDiv(curr['r'], curr['r2']))),
        Equals(prox['s'], curr['s2']),
        Equals(prox['s2'], BVSub(curr['s'], BVUDiv(curr['s'], curr['s2']))),
        Equals(prox['t'], curr['t2']),
        Equals(prox['t2'], BVSub(curr['t'], BVUDiv(curr['t'], curr['t2']))),
        Equals(curr['q'], BVUDiv(curr['r'], curr['r2']))  # Unsigned division
    )

    erro = And(
        Equals(curr['pc'], Int(1)),
        Equals(curr['r'], BV(0, n)),
        Equals(prox['pc'], Int(3))
    )

    erro_overflow = And(
        Equals(curr['pc'], Int(1)),
        Or(
            BVUGT(curr['r'], BV(2**n - 1, n)),   # Unsigned upper bound
            BVUGT(curr['r2'], BV(2**n - 1, n)),
            BVUGT(curr['s'], BV(2**n - 1, n)),
            BVUGT(curr['s2'], BV(2**n - 1, n)),
            BVUGT(curr['t'], BV(2**n - 1, n)),
            BVUGT(curr['t2'], BV(2**n - 1, n)),
            BVUGT(curr['q'], BV(2**n - 1, n))
        ),
        Equals(prox['pc'], Int(3))
    )

    final = And(
        Equals(curr['pc'], Int(1)),
        Equals(curr['r2'], BV(0, n)),
        Equals(prox['pc'], Int(2)),
        Equals(prox['r'], curr['r']),
        Equals(prox['s'], curr['s']),
        Equals(prox['t'], curr['t']),
        Equals(prox['r2'], curr['r2']),
        Equals(prox['s2'], curr['s2']),
        Equals(prox['t2'], curr['t2']),
        Equals(prox['q'], curr['q']),
    )
    return Or(t_main, erro, erro_overflow, final)

def genTrace(vars, init, trans, error, n, N):
    X = [genState(vars, 'X', i, n) for i in range(N+1)]
    a = Symbol('a', BVType(n))
    b = Symbol('b', BVType(n))

    I = init(X[0], a, b, n)
    traces = [trans(X[i], X[i+1]) for i in range(N)]
    final_state = Equals(X[N]['pc'], Int(2))  # Estado final esperado

    with Solver(name="z3") as solver:
        if solver.solve([I, And(traces), final_state]):
            model = solver.get_model()
            print("> Traço encontrado, o sistema é SEGURO")
            for i in range(N+1):
                print(f"Estado X{i}:")
                for v in X[i]:
                    value = model[X[i][v]].constant_value()
                    print(f"        {v} = {value}")
        else:
            print("> Nenhum traço encontrado, o sistema pode ser INSEGURO")



# Similarly, update the `genState2`, `init2`, `error2`, and `trans2` functions to use signed operations.

def model_checking(vars, init, trans, error, bits, N, M):
    with Solver(name="z3") as solver:
        a = Symbol('a', BVType(bits))
        b = Symbol('b', BVType(bits))
        X = [genState2(vars, "X", i, bits) for i in range(N+1)]
        Y = [genState2(vars, "Y", i, bits) for i in range(M+1)]
        transt = invert(trans)

        order = sorted([(x, y) for x in range(1, N+1) for y in range(1, M+1)], key=lambda tup: tup[0] + tup[1])

        for (n, m) in order:
            I = init(X[0], a, b, bits)
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])
            Rn = And(I, Tn)

            E = error(Y[0], bits)
            Bm = And([transt(Y[i], Y[i+1]) for i in range(m)])
            Um = And(E, Bm)

            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro")
                return
            else:
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B)
                if C is None:
                    print("> O interpolante é None")
                    break
                
                C0 = rename(C, X[0])
                T = trans(X[0], X[1])
                C1 = rename(C, X[1])

                if not solver.solve([C0, T, Not(C1)]):
                    print("> O sistema é seguro")
                    return
                else:
                    S = rename(C, X[n])
                    while True:
                        T = trans(X[n], Y[m])
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante")
                            break
                        else:
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                                print("> O sistema é seguro")
                                return
                            else:
                                S = Or(S, Cn)

    print("> Não foi provada a segurança ou insegurança do sistema")

vars = ['r', 'r2', 's', 's2', 't', 't2']
genTrace(vars, init, trans1, error1, 8, 8)
vars = ['r', 'r2', 's', 's2', 't', 't2', 'pc2']
model_checking(vars, init2, trans2, error2, 8, 8, 8)

> Traço encontrado, o sistema é SEGURO
Estado X0:
        r = 10
        r2 = 2
        s = 1
        s2 = 0
        t = 0
        t2 = 1
        pc = 0
        q = 5
Estado X1:
        r = 2
        r2 = 5
        s = 0
        s2 = 2
        t = 1
        t2 = 0
        pc = 1
        q = 0
Estado X2:
        r = 5
        r2 = 2
        s = 2
        s2 = 0
        t = 0
        t2 = 2
        pc = 0
        q = 2
Estado X3:
        r = 2
        r2 = 3
        s = 0
        s2 = 3
        t = 2
        t2 = 0
        pc = 1
        q = 0
Estado X4:
        r = 3
        r2 = 2
        s = 3
        s2 = 0
        t = 0
        t2 = 3
        pc = 0
        q = 1
Estado X5:
        r = 2
        r2 = 2
        s = 0
        s2 = 4
        t = 3
        t2 = 0
        pc = 1
        q = 1
Estado X6:
        r = 2
        r2 = 1
        s = 4
        s2 = 0
        t = 0
        t2 = 4
        pc = 0
        q = 2
Estado X7:
        r = 1
        r2 = 0
        s = 0
        s2 = 5
  